In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import json

In [2]:
PATH = "C:\\Users\\Victor\\Downloads\\chromedriver_win32\\chromedriver" # Change this file path to match the chromedriver location on your device!
driver = webdriver.Chrome(PATH)

C:\Users\Victor\AppData\Local\Temp/ipykernel_20848/1940386753.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [3]:
search_terms = [
    'beef stew',
    'salad',
    'pasta',
    'steak',
    'fish',
    'smoothie',
    'dessert',]

data = []


# Replace XXX with the search term and offset=i
SEARCH_URL = "https://www.allrecipes.com/search?XXX=XXX&q=XXX&offset=0"


In [4]:
def clean_field(entry_data, s):
    l = s.rsplit(' ', 1)
    entry_data[l[0]] = int(''.join(c for c in l[1] if c.isdigit()))
    # string = r'data-doc-id="[0-9]+'
    # match = re.search(string, page.text)

# Search for search_term and retrieve at most n recipes
def scrape(search_term, n):
    
    i = 0
    while True:
        
        i += 1
        if i == n:
            break
        
        try:
            # Modify search url
            search_term = search_term.replace(" ", "%20")
            url = SEARCH_URL.replace("0", str(i))
            url = url.replace("XXX", search_term)

            driver.get(url)
            # //*[@id="mntl-card-list-items_1-0"]
            link = driver.find_element(By.XPATH, '//*[@id="mntl-card-list-items_1-0"]')

            # Make sure the link is actually a recipe and not a dumb blogpost smh
            u = link.get_attribute('href')      
            if "/recipe/" not in u:
                n += 1
                continue
            print(u)
            
            # Click on link
            link.click()
            
            # Holds the data on the current recipe entry
            entry_data = {}
            
            # Extract recipe name and rating
            entry_data['name'] = driver.find_element(By.XPATH, '//*[@id="article-heading_1-0"]').text
            try:
                entry_data['rating'] = driver.find_element(By.XPATH, '//*[@id="mntl-recipe-review-bar__rating_1-0"]').text
            except Exception as e:
                entry_data['rating'] = None
                
            # Click on "show nutrition info" button
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mntl-nutrition-facts-label_1-0"]/button'))).click()

            # Extract the nutrition data
            entry_data['calories'] = driver.find_element(By.XPATH, '//*[@id="mntl-nutrition-facts-label_1-0"]/div/div/table/thead/tr[3]/th/span[2]').text
            entry_data['servings'] = driver.find_element(By.XPATH, '//*[@id="mntl-nutrition-facts-label_1-0"]/div/div/table/thead/tr[2]/th/span[2]').text
            
            NUTRITION_FIELD =  '//*[@id="mntl-nutrition-facts-label_1-0"]/div/div/table/tbody/tr[XXX]/td[1]'
            
            j = 2
            while True:
                try:
                    nutrition_field = NUTRITION_FIELD.replace('XXX', str(j))
                    clean_field(entry_data, driver.find_element(By.XPATH, nutrition_field).text)
                except Exception as e:
                    break
                j = j+1
            
            print(entry_data)
            data.append(entry_data)
            
        except:
            print("Ran into an error. Proceeding...")
            continue
        

In [5]:
for s in search_terms:
    print("Scraping for " + s)
    scrape(s, 10)

Scraping for beef stew
https://www.allrecipes.com/recipe/8369933/greek-style-beef-stew-stifado/
{'name': 'Greek-Style Beef Stew (Stifado)', 'rating': '4.8', 'calories': '777', 'servings': '6', 'Total Fat': 55, 'Saturated Fat': 20, 'Cholesterol': 163, 'Sodium': 1519, 'Total Carbohydrate': 20, 'Dietary Fiber': 2, 'Protein': 42, 'Potassium': 1073}
https://www.allrecipes.com/recipe/8472977/got-no-beef-stew/
{'name': 'Got No Beef Stew', 'rating': '5.0', 'calories': '408', 'servings': '4', 'Total Fat': 15, 'Saturated Fat': 5, 'Cholesterol': 15, 'Sodium': 2541, 'Total Carbohydrate': 60, 'Dietary Fiber': 8, 'Protein': 13, 'Potassium': 1765}
https://www.allrecipes.com/recipe/285607/hearty-spanish-beef-stew/
{'name': 'Hearty Spanish Beef Stew', 'rating': '5.0', 'calories': '305', 'servings': '4', 'Total Fat': 18, 'Saturated Fat': 5, 'Cholesterol': 67, 'Sodium': 742, 'Total Carbohydrate': 12, 'Dietary Fiber': 2, 'Total Sugars': 5, 'Protein': 23, 'Vitamin C': 49, 'Calcium': 74, 'Iron': 6, 'Potassi

https://www.allrecipes.com/recipe/23404/tomato-cream-sauce-for-pasta/
{'name': 'Tomato-Cream Sauce for Pasta', 'rating': '4.6', 'calories': '182', 'servings': '5', 'Total Fat': 17, 'Saturated Fat': 8, 'Cholesterol': 39, 'Sodium': 270, 'Total Carbohydrate': 7, 'Dietary Fiber': 1, 'Total Sugars': 4, 'Protein': 2, 'Vitamin C': 8, 'Calcium': 64, 'Iron': 2, 'Potassium': 180}
https://www.allrecipes.com/recipe/254895/homemade-spinach-pasta/
{'name': 'Homemade Spinach Pasta', 'rating': '4.9', 'calories': '568', 'servings': '2', 'Total Fat': 10, 'Saturated Fat': 2, 'Cholesterol': 186, 'Sodium': 586, 'Total Carbohydrate': 97, 'Dietary Fiber': 4, 'Total Sugars': 1, 'Protein': 20, 'Vitamin C': 12, 'Calcium': 87, 'Iron': 8, 'Potassium': 435}
https://www.allrecipes.com/recipe/8354778/clam-chowder-shell-pasta/
{'name': 'Clam Chowder Shell Pasta', 'rating': '5.0', 'calories': '570', 'servings': '4', 'Total Fat': 30, 'Saturated Fat': 14, 'Cholesterol': 116, 'Sodium': 989, 'Total Carbohydrate': 53, 'Die

https://www.allrecipes.com/recipe/138159/pineapple-and-banana-smoothie/
{'name': 'Pineapple and Banana Smoothie', 'rating': '4.6', 'calories': '313', 'servings': '1', 'Total Fat': 1, 'Saturated Fat': 0, 'Sodium': 10, 'Total Carbohydrate': 79, 'Dietary Fiber': 6, 'Total Sugars': 53, 'Protein': 3, 'Vitamin C': 93, 'Calcium': 57, 'Iron': 2, 'Potassium': 941}
https://www.allrecipes.com/recipe/161345/strawberry-banana-smoothie/
{'name': 'Strawberry Banana Smoothie', 'rating': '4.4', 'calories': '332', 'servings': '2', 'Total Fat': 3, 'Saturated Fat': 2, 'Cholesterol': 9, 'Sodium': 125, 'Total Carbohydrate': 68, 'Dietary Fiber': 5, 'Total Sugars': 50, 'Protein': 12, 'Vitamin C': 27, 'Calcium': 330, 'Iron': 1, 'Potassium': 949}
https://www.allrecipes.com/recipe/282563/my-green-protein-smoothie/
{'name': 'My Green Protein Smoothie', 'rating': None, 'calories': '336', 'servings': '2', 'Total Fat': 2, 'Saturated Fat': 1, 'Cholesterol': 6, 'Sodium': 215, 'Total Carbohydrate': 50, 'Dietary Fiber':

In [10]:
data

[{'name': 'Greek-Style Beef Stew (Stifado)',
  'rating': '4.8',
  'calories': '777',
  'servings': '6',
  'Total Fat': 55,
  'Saturated Fat': 20,
  'Cholesterol': 163,
  'Sodium': 1519,
  'Total Carbohydrate': 20,
  'Dietary Fiber': 2,
  'Protein': 42,
  'Potassium': 1073},
 {'name': 'Got No Beef Stew',
  'rating': '5.0',
  'calories': '408',
  'servings': '4',
  'Total Fat': 15,
  'Saturated Fat': 5,
  'Cholesterol': 15,
  'Sodium': 2541,
  'Total Carbohydrate': 60,
  'Dietary Fiber': 8,
  'Protein': 13,
  'Potassium': 1765},
 {'name': 'Hearty Spanish Beef Stew',
  'rating': '5.0',
  'calories': '305',
  'servings': '4',
  'Total Fat': 18,
  'Saturated Fat': 5,
  'Cholesterol': 67,
  'Sodium': 742,
  'Total Carbohydrate': 12,
  'Dietary Fiber': 2,
  'Total Sugars': 5,
  'Protein': 23,
  'Vitamin C': 49,
  'Calcium': 74,
  'Iron': 6,
  'Potassium': 524},
 {'name': 'Rutabaga Beef Stew',
  'rating': '4.3',
  'calories': '462',
  'servings': '4',
  'Total Fat': 31,
  'Saturated Fat': 10,


In [13]:
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)